In [1]:
import pandas as pd
import requests
import time

In [2]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0",
    "Accept-Language": 'en-US,en;q=0.9',
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Referer": "https://tiki.vn/",
    "From": "",
    "af-ac-enc-dat": "",
    "x-api-source": "pc"
}

In [3]:
URL = "https://api.tiki.vn/raiden/v2/menu-config?platform=desktop"

In [4]:
response = requests.get(URL, headers=HEADERS)
data = response.json()

In [5]:
groups = data["menu_block"]["items"]
group_list = []

for group in groups:
    text = group["text"]
    link = group["link"]
    # Extract category ID from the link
    group_id = link.split("/")[-1][1:]  
    group_list.append([group_id, text])

group_df = pd.DataFrame(group_list, columns=["Group_ID", "Group_Name"])

In [6]:
URL = "https://tiki.vn/api/personalish/v1/blocks/listings"

In [7]:
def fetch_total_units_sold(group_id, page):
    PARAMS = {"category": group_id, "page": page}
    response = requests.get(URL, headers=HEADERS, params=PARAMS)
    data = response.json()

    total_units_sold = 0

    if 'data' in data:
        # Iterate through the data list
        for item in data['data']:
            if 'quantity_sold' in item and isinstance(item['quantity_sold'], dict) and 'value' in item['quantity_sold']:
                # Add the 'value' to the total units sold
                total_units_sold += item['quantity_sold']['value']

    return total_units_sold

In [8]:
# Iterate over groups to get total units sold
for _, row in group_df.iterrows():
    group_id = row["Group_ID"]
    group_name = row["Group_Name"]
    
    total_units_sold = 0

    # Fetch total number of items for the current group 
    PARAMS = {"category": group_id, "page": 1}
    response = requests.get(URL, headers=HEADERS, params=PARAMS)
    data = response.json()
    last_page = data.get("paging", {}).get("last_page", 1)

    for page in range(1, last_page + 1):
        total_units_sold += fetch_total_units_sold(group_id, page)
        time.sleep(3)

    print(f"Total units sold for ID:{group_id} - {group_name}: {total_units_sold}")

Total units sold for ID:8322 - Nhà Sách Tiki: 5995718
Total units sold for ID:1883 - Nhà Cửa - Đời Sống: 1592658
Total units sold for ID:1789 - Điện Thoại - Máy Tính Bảng: 60164
Total units sold for ID:2549 - Đồ Chơi - Mẹ & Bé: 2575949
Total units sold for ID:1815 - Thiết Bị Số - Phụ Kiện Số: 661903
Total units sold for ID:1882 - Điện Gia Dụng: 868028
Total units sold for ID:1520 - Làm Đẹp - Sức Khỏe: 2678523
Total units sold for ID:8594 - Ô Tô - Xe Máy - Xe Đạp: 244510
Total units sold for ID:931 - Thời trang nữ: 170353
Total units sold for ID:4384 - Bách Hóa Online: 2372692
Total units sold for ID:1975 - Thể Thao - Dã Ngoại: 374503
Total units sold for ID:915 - Thời trang nam: 304324
Total units sold for ID:17166 - Cross Border - Hàng Quốc Tế: 10227
Total units sold for ID:1846 - Laptop - Máy Vi Tính - Linh kiện: 252823
Total units sold for ID:1686 - Giày - Dép nam: 81204
Total units sold for ID:4221 - Điện Tử - Điện Lạnh: 41730
Total units sold for ID:1703 - Giày - Dép nữ: 96948
Tot

Based on the data of total units sold for product categories, here's some analysis:

- **Insights on High Performing Categories**: Identify and highlight categories with significantly high sales numbers. 
Categories like **Nhà Sách Tiki** and **Đồ Chơi - Mẹ & Bé** stand out with remarkably high total units sold, indicating strong consumer demand and market presence. **Nhà Sách Tiki** has sold a total of **5,995,718** units, while **Đồ Chơi - Mẹ & Bé** has sold 2,575,949 units.


- **Identify Low Performing Categories**: Highlight categories with relatively low sales numbers. 
**Cross Border - Hàng Quốc Tế** and **Điện Tử - Điện Lạnh** have significantly lower sales compared to other categories, indicating potential areas for improvement or strategic focus.
